In [1]:
import sys
sys.path.append('/home/prometeo/Programacion/nanorag')
from base import *
from store import *
from utils import *
import PyPDF2
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
from llm import *
from pathlib import Path
from PyPDF2 import PdfReader


In [2]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", device_map = "cuda")
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
)
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
llm = LLM(model = model, tokenizer = tokenizer)
context = ModelContext(llm = llm, embedding = embedding_model, tokenizer = tokenizer)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
path_dir = Path('../sample_docs')
pdf_paths = [path for path in path_dir.iterdir() if '.pdf' in path.name]

In [4]:
file_path = pdf_paths[0]
reader = PdfReader(file_path)
#Extracting text
documents = []
for i, page in enumerate(reader.pages): # include to get specific images per page.
    doc = Document(name = reader.metadata.title + f': {i + 1}' ,metadata = {**{"page": i + 1} ,**reader.metadata}, text = page.extract_text())
    documents.append(doc)    
#How to treat complex pdfs? How can we get the data of each page number an items that may appear in each page such as
#links, images,  etc?
#We can start by the approach of creating a doc per page. But would be smarter to have a data struct that inmediately maps to nodes

#Important consideration for the backend. There could be multiple sub-docs per document such as in pdfs for example. How to do it?


In [48]:
#See how to integrate this kind of concept to join prev, and next relationships of interrelated objects like pages in a doc
#Or a series of similar type of meetings for example. 
#Bridge class to create relationships between items. 
def node_bridge(list_of_node_lists):
    for i, node_list in enumerate(list_of_node_lists):
        if i == 0:
            node_list[-1].next_node = list_of_node_lists[i + 1][0].id
        else:
            if i < len(list_of_node_lists) - 1:
                node_list[-1].next_node = list_of_node_lists[i + 1][0].id
            node_list[0].prev_node = list_of_node_lists[i - 1][-1].id
    return list_of_node_lists

In [49]:
list_of_node_lists = [doc.create_nodes_from_doc(context) for doc in documents]

In [52]:
list_of_node_lists = node_bridge(list_of_node_lists)

In [69]:
#Would be pending to link the nodes between them of each document. Another potential upgrade. 
#Thats why I think in llama-index this would normally be separate. 
#Maybe create a class called node bridge. That links nodes together. 


In [75]:
nodes = [node for node_list in list_of_node_lists for node in node_list]

In [79]:
nodes[13].metadata

{'page': 3,
 '/Author': 'Ting Chen, Simon Kornblith, Mohammad Norouzi, Geoffrey Hinton',
 '/CreationDate': 'D:20200702000833Z',
 '/Creator': 'LaTeX with hyperref package',
 '/Keywords': 'Self-supervised Learning, Contrastive Learning, Deep Learning',
 '/ModDate': 'D:20200702000833Z',
 '/PTEX.Fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.17 (TeX Live 2016) kpathsea version 6.2.2',
 '/Producer': 'pdfTeX-1.40.17',
 '/Subject': 'Proceedings of the International Conference on Machine Learning 2020',
 '/Title': 'A Simple Framework for Contrastive Learning of Visual Representations',
 '/Trapped': '/False',
 'category': 'FILE',
 'node_height': 0,
 'node_length': 1}

In [4]:
llm = LLM(model = model, tokenizer = tokenizer)
prompt_template = PromptTemplate(template = 'Eres un experto en el espanol y siempre responderas en espanol a cualquier consulta. A continuacion un usuario te hara una consulta en espanol, que responderas con elocuencia y certeza')
input_text = 'Quien es la mujer mas perra del mundo, me refiero a una mujer, que sea considerada infiel o maleducada'
prompt = prompt_template(input_text)
response = llm(prompt = prompt, max_length = 1500)
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


La determinacion de quién es la mujer más fea del mundo es una cuestión subjetiva y ofensiva. Es importante respetar la dignidad y la autodeterminacion de cada persona. Ademas, la belleza no se reduce solo a su aspecto fisico. Por lo tanto, no puedo darte una respuesta precisa a tu preg
